In [1]:
import os
import tensorflow as tf
import numpy as np 

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.chdir('/home/aditya/Downloads/TensorflowDatasets/train')

### Extracting Filenames and Classes

In [3]:
classes=[]
images=[]
for i in os.listdir('/home/aditya/Downloads/TensorflowDatasets/train'):
    filename=tf.read_file(i)
    images.append(filename)
    if i[0]=='c':
        classes.append(0)
    else:
        classes.append(1)

### One_hot encoding of Classes

In [4]:
y_true=tf.one_hot(classes,2)

In [5]:
images[1]

<tf.Tensor 'ReadFile_1:0' shape=() dtype=string>

### Creating Training(24000 images) and Validation Datasets(1000 images)

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((images[:24000], y_true[:24000]))
validation_dataset = tf.data.Dataset.from_tensor_slices((images[24000:25000], y_true[24000:25000]))

In [7]:
len(images)

25000

In [8]:
dataset

<TensorSliceDataset shapes: ((), (2,)), types: (tf.string, tf.float32)>

### Getting Images from Filenames

In [9]:
def parse_function(images, y_true):
    image = tf.image.decode_jpeg(images, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_images(image, [64, 64])
    return image, y_true

In [10]:
dataset = dataset.map(parse_function, num_parallel_calls=4)
validation_dataset = validation_dataset.map(parse_function, num_parallel_calls=4)

### Augmentation(Flip_up_down,Flip_Left_Right,Brightness,Saturation) with 50% chance. 

In [11]:
def train_preprocess(image, y_true):
    a=np.random.randint(1,5)
    b=np.random.randint(1,3)
    if b==1:
        image=image
    else:
        if a==1:
            image = tf.image.random_flip_left_right(image)
        elif a==2:
            image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        elif a==3:
            image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        else:
            image = tf.image.random_flip_up_down(image)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image,y_true

In [12]:
dataset = dataset.map(train_preprocess, num_parallel_calls=2)

In [13]:
dataset

<ParallelMapDataset shapes: ((64, 64, 3), (2,)), types: (tf.float32, tf.float32)>

### Shuffling Dataset

In [14]:
dataset=dataset.shuffle(24000)

### Batchsize=16

In [15]:
dataset=dataset.batch(16)

In [16]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

### Reinitializable Iterator

In [17]:
iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
X,y = iterator.get_next()

In [18]:
y

<tf.Tensor 'IteratorGetNext:1' shape=(?, 2) dtype=float32>

In [19]:
os.getcwd()

'/home/aditya/Downloads/TensorflowDatasets/train'

In [20]:
dataset

<BatchDataset shapes: ((?, 64, 64, 3), (?, 2)), types: (tf.float32, tf.float32)>

In [21]:
X=tf.reshape(X,[-1,64,64,3])

### VGG 1st Layer + 1 Fully Connected into Prediction

In [22]:
conv1=tf.layers.conv2d(X,64,3)
conv2=tf.layers.conv2d(conv1,64,3)
maxpool1=tf.layers.max_pooling2d(conv2,[2,2],[2,2])
flat=tf.layers.flatten(maxpool1)
dense1=tf.layers.dense(flat,1024)
y_pred=tf.layers.dense(dense1,2)

In [23]:
y_pred

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 2) dtype=float32>

In [24]:
y_true

<tf.Tensor 'one_hot:0' shape=(25000, 2) dtype=float32>

### Initializing Iterator

In [25]:
validation_dataset=validation_dataset.batch(16)
training_init_op = iterator.make_initializer(dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

### Training and Loss(Softmax)

In [26]:
loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))
train_op=tf.train.AdamOptimizer().minimize(loss_op) 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy 

In [27]:
prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Session

In [28]:
 with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(100):
        sess.run(training_init_op)        
        print(sess.run([train_op,loss_op,accuracy]))
        sess.run(validation_init_op)
        print('Validation Batch Accuracy',sess.run(accuracy))

[None, 0.74075466, 0.375]
Validation Batch Accuracy 0.625
[None, 181.98828, 0.4375]
Validation Batch Accuracy 0.625
[None, 22.696287, 0.5]
Validation Batch Accuracy 0.375
[None, 106.50009, 0.5]
Validation Batch Accuracy 0.375
[None, 93.057144, 0.4375]
Validation Batch Accuracy 0.375
[None, 5.702729, 0.5]
Validation Batch Accuracy 0.625
[None, 102.00393, 0.4375]
Validation Batch Accuracy 0.625
[None, 131.04788, 0.5]
Validation Batch Accuracy 0.625
[None, 94.51536, 0.4375]
Validation Batch Accuracy 0.625
[None, 61.685844, 0.375]
Validation Batch Accuracy 0.625
[None, 11.103902, 0.5]
Validation Batch Accuracy 0.375
[None, 28.072628, 0.375]
Validation Batch Accuracy 0.375
[None, 26.324997, 0.5625]
Validation Batch Accuracy 0.375
[None, 39.734818, 0.3125]
Validation Batch Accuracy 0.5
[None, 1.1171906, 0.6875]
Validation Batch Accuracy 0.625
[None, 27.599487, 0.4375]
Validation Batch Accuracy 0.625
[None, 23.479893, 0.6875]
Validation Batch Accuracy 0.625
[None, 28.643478, 0.625]
Validation

In [29]:
X.shape

TensorShape([Dimension(None), Dimension(64), Dimension(64), Dimension(3)])

In [30]:
y.shape

TensorShape([Dimension(None), Dimension(2)])

In [31]:
y_pred.shape

TensorShape([Dimension(None), Dimension(2)])